In [36]:
from scipy.fft import rfft, irfft
import scipy.signal as ssig
import sounddevice as sd
import numpy as np
import librosa

In [37]:
wav, sample_rate = librosa.load(path="hw2assets/collectathon.wav", mono=True, dtype=np.float32)


In [38]:
sd.play(data=wav, samplerate=sample_rate)
sd.sleep(2000) # sleep for 2 seconds to just test play
sd.stop() # stop play once sleep is done.

In [ ]:
blockSizes = [64] #, 256, 512, 1024, 4096]

for blockSize in blockSizes:

    print(f"Playing audio with block size: {blockSize}")
    # sd.play(wav_blocks, samplerate=sample_rate)
    # sd.sleep(2000)
    # sd.stop()

NameError: name 'samples' is not defined